# TASKS

- The data entering the stage should be normalized and cleaned (This can be done using a store procedure from raw to stage or using a python script on the files before their insertion into the stage)
- Decide which tables are dimensional and which tables are fact tables
- Stage tables should be transferred to the hist layer using store procedures
- Dimensional tables should be type 2 slowly changing dimensions
- Create 3 triggers on after update, after insert and after delete on any tables in the data warehouse
- Create 3 views on the hist layer of the data warehouse that filter data
- Create 3 views on the hist layer of the data warehouse that aggregate data
- Create 3 different store procedures that perform a particular task in the data warehouse

## Setup Database Connection

Using MariaDB instead of MySQL due to operating system compatibility issues. These are entirely interchangeable due to MariaDB being a fork of MySQL with the commitment to maintain compatibility.

In [1]:
import mariadb
import pandas as pd
import numpy as np

try:
    conn = mariadb.connect(
        user='root',
        unix_socket="mysql/mysql.sock"
    )
    cursor = conn.cursor()
except mariadb.Error as e:
    print(f"Error connecting to MariaDB: {e}")


## Create Database & Schema

In [2]:
cursor.execute("CREATE DATABASE IF NOT EXISTS AnimeDataWarehouse;")
cursor.execute("USE AnimeDataWarehouse;")

cursor.execute("CREATE SCHEMA IF NOT EXISTS stage;")
cursor.execute("CREATE SCHEMA IF NOT EXISTS hist;")

## Create Stage Tables

In [3]:
# Define table for anime data
anime_table_query = """
CREATE TABLE IF NOT EXISTS stage.DimAnime (
    MAL_ID INT PRIMARY KEY,
    Name VARCHAR(255),
    English_name VARCHAR(255),
    Japanese_name VARCHAR(255),
    Type VARCHAR(50),
    Episodes INT,
    Aired VARCHAR(100),
    Premiered VARCHAR(50),
    Producers TEXT,
    Licensors TEXT,
    Studios TEXT,
    Source VARCHAR(50),
    Duration INT,  -- Duration in seconds
    Rating VARCHAR(50),
    Score FLOAT,
    Ranked INT,
    Popularity INT,
    Members INT,
    Favorites INT,
    Watching INT,
    Completed INT,
    OnHold INT,
    Dropped INT,
    PlanToWatch INT
);
"""
cursor.execute(anime_table_query)
conn.commit()


In [4]:
# Define table for user data
user_table_query = """
CREATE TABLE IF NOT EXISTS stage.DimUser (
    user_id INT PRIMARY KEY
);
"""
cursor.execute(user_table_query)
conn.commit()


In [5]:
# Define fact table for user-anime interactions
fact_table_query = """
CREATE TABLE IF NOT EXISTS stage.FactUserAnimeInteractions (
    user_id INT,
    anime_id INT,
    rating INT,
    watching_status INT,
    watched_episodes INT,
    PRIMARY KEY (user_id, anime_id)
);
"""
cursor.execute(fact_table_query)
conn.commit()


In [6]:
# Define table for watching statuses
status_table_query = """
CREATE TABLE IF NOT EXISTS stage.DimWatchingStatus (
    status INT PRIMARY KEY,
    description VARCHAR(100)
);
"""
cursor.execute(status_table_query)
conn.commit()


In [7]:
# Define table for genres
genre_table_query = """
CREATE TABLE IF NOT EXISTS stage.DimGenre (
    genre_id INT PRIMARY KEY AUTO_INCREMENT,
    genre_name VARCHAR(100)
);
"""
cursor.execute(genre_table_query)
conn.commit()

# Define bridge table between anime and genres
bridge_genre_query = """
CREATE TABLE IF NOT EXISTS stage.BridgeAnimeGenre (
    anime_id INT,
    genre_id INT,
    PRIMARY KEY (anime_id, genre_id)
);
"""
cursor.execute(bridge_genre_query)
conn.commit()


In [8]:
# Define table for studios
studio_table_query = """
CREATE TABLE IF NOT EXISTS stage.DimStudio (
    studio_id INT PRIMARY KEY AUTO_INCREMENT,
    studio_name VARCHAR(255)
);
"""
cursor.execute(studio_table_query)
conn.commit()

# Define bridge table between anime and studios
bridge_studio_query = """
CREATE TABLE IF NOT EXISTS stage.BridgeAnimeStudio (
    anime_id INT,
    studio_id INT,
    PRIMARY KEY (anime_id, studio_id)
);
"""
cursor.execute(bridge_studio_query)
conn.commit()


In [9]:
tables = [
    'DimAnime', 'DimUser', 'FactUserAnimeInteractions', 
    'DimWatchingStatus', 'DimGenre', 'BridgeAnimeGenre', 
    'DimStudio', 'BridgeAnimeStudio'
]

def sanity_check_tables(cursor, tables):
    for table in tables:
        print(f"\nChecking table: {table}")

        cursor.execute(f"DESCRIBE stage.{table};")
        schema = cursor.fetchall()
        print(f"Schema of {table}:")
        for col in schema:
            print(f"  {col[0]} ({col[1]})")
        
        cursor.execute(f"SELECT COUNT(*) FROM stage.{table};")
        row_count = cursor.fetchone()[0]
        print(f"Number of rows in {table}: {row_count}")

        cursor.execute(f"SELECT * FROM stage.{table} LIMIT 5;")
        rows = cursor.fetchall()
        if rows:
            print(f"Sample rows from {table}:")
            for row in rows:
                print(f"  {row}")
        else:
            print(f"No rows in {table}.")

sanity_check_tables(cursor, tables)



Checking table: DimAnime
Schema of DimAnime:
  MAL_ID (int(11))
  Name (varchar(255))
  English_name (varchar(255))
  Japanese_name (varchar(255))
  Type (varchar(50))
  Episodes (int(11))
  Aired (varchar(100))
  Premiered (varchar(50))
  Producers (text)
  Licensors (text)
  Studios (text)
  Source (varchar(50))
  Duration (int(11))
  Rating (varchar(50))
  Score (float)
  Ranked (int(11))
  Popularity (int(11))
  Members (int(11))
  Favorites (int(11))
  Watching (int(11))
  Completed (int(11))
  OnHold (int(11))
  Dropped (int(11))
  PlanToWatch (int(11))
Number of rows in DimAnime: 17562
Sample rows from DimAnime:
  (1, 'Cowboy Bebop', 'Cowboy Bebop', 'カウボーイビバップ', 'TV', 26, 'Apr 3, 1998 to Apr 24, 1999', 'Spring 1998', 'Bandai Visual', 'Funimation, Bandai Entertainment', 'Sunrise', 'Original', 1440, 'R - 17+ (violence & profanity)', 8.78, 28, 39, 1251960, 61971, 105808, 718161, 71513, 26678, 329800)
  (5, 'Cowboy Bebop: Tengoku no Tobira', 'Cowboy Bebop:The Movie', 'カウボーイビバップ 天国の

# Data Exploration, Cleaning and Loading

We are doing all these steps one file at a time to account for memory problems when transforming and loading the data (animelist is the biggest issue here)

In [10]:
# Preliminary imports
import numpy as np
import pandas as pd
import re

We are using pandas to load the raw data and clean it in preparation for insertion into the stage tables.

To understand the data better, we will explore:
- Data type of each column (unlikely to be useful, pandas tends to cast to a generic object type because of non-standard data)
- Number of null/NaN/Unknown values in each column
- Unique values in each column (top 5 most common values)

We're putting them in a table to make it easier to read.

### anime.csv

#### Exploring anime.csv

In [11]:
anime_df = pd.read_csv('raw/anime.csv')
print("Columns in anime_df:")
print(anime_df.columns)

Columns in anime_df:
Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')


In [12]:
# Check for null/NAN or Unknown values
analysis_anime_df = pd.DataFrame({
    'Data Type': anime_df.dtypes,
    'Null': anime_df.isnull().sum(),
    'NAN': anime_df.isna().sum(),
    'Unknown': anime_df.isin(['Unknown']).sum(),
    'Most Common Unique Values': [anime_df[column].value_counts().index[:5].tolist() for column in anime_df.columns],
})

analysis_anime_df

,Data Type,Null,NAN,Unknown,Most Common Unique Values
MAL_ID,int64,0,0,0,"[48492, 1, 5, 6, 7]"
Name,object,0,0,0,[Maou Gakuin no Futekigousha: Shijou Saikyou n...
Score,object,0,0,5141,"[Unknown, 6.48, 6.3, 6.31, 6.52]"
Genres,object,0,0,63,"[Hentai, Music, Comedy, Kids, Kids, Music]"
English name,object,0,0,10565,"[Unknown, Cyborg 009, Meow Meow Japanese Histo..."
Japanese name,object,0,0,48,"[Unknown, ゲゲゲの鬼太郎, 僕のヒーローアカデミア, おしりたんてい, 宇宙の騎士..."
Type,object,0,0,37,"[TV, OVA, Movie, Special, ONA]"
Episodes,object,0,0,516,"[1, 12, 2, 13, 3]"
Aired,object,0,0,309,"[Unknown, 2005, 2004, 2003, 2021 to ?]"
Premiered,object,0,0,12817,"[Unknown, Spring 2017, Fall 2016, Spring 2018,..."


#### Cleaning anime.csv

The critical columns MAL_ID and Name do not have any missing values. The other columns have missing data noted by 'Unknown'. Score-# columns are floats, but also have 'Unknown' values: Score-# indicates number of users who rated the anime (cannot be fractional). Duration is a string written as 'Unknown' or '24 min. per ep.'.

##### Clean Numeric Columns

In [13]:
anime_df_cleaned = anime_df.copy()

# Convert 'Unknown' to NaN in columns that should be numeric
num_cols = ['Score', 'Episodes', 'Ranked', 'Popularity', 'Members', 'Favorites', 
            'Watching', 'Completed', 'On-Hold', 'Dropped', 'Plan to Watch']

# Replace 'Unknown' with NaN, then convert to numeric
anime_df_cleaned[num_cols] = anime_df_cleaned[num_cols].replace('Unknown', pd.NA).apply(pd.to_numeric, errors='coerce')

# We are not setting NaN values to 0 because it will skew the data, better make that
# decision later depending on what is needed.

##### Clean String Columns

In [14]:
# Replace 'Unknown' with NaN in string columns
str_cols = ['Genres', 'English name', 'Japanese name', 'Type', 'Aired', 'Premiered', 
            'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating']

anime_df_cleaned[str_cols] = anime_df_cleaned[str_cols].replace('Unknown', pd.NA)

# Strip whitespace from all string columns
anime_df_cleaned[str_cols] = anime_df_cleaned[str_cols].apply(lambda x: x.str.strip() if x.dtype == "object" else x)


#### Handling Duration

In [15]:
# Show unique values in anime_df_cleaned duration
anime_df_cleaned['Duration'].unique()

array(['24 min. per ep.', '1 hr. 55 min.', '25 min. per ep.',
       '23 min. per ep.', '27 min. per ep.', '24 min.', '22 min. per ep.',
       '1 hr. 44 min.', '1 hr. 27 min.', '1 hr. 22 min.',
       '30 min. per ep.', '1 hr. 31 min.', '2 hr. 4 min.',
       '1 hr. 45 min.', '52 min.', '1 hr. 59 min.', '2 hr.',
       '21 min. per ep.', '14 min. per ep.', '12 min. per ep.',
       '34 min. per ep.', '46 min. per ep.', '1 hr. 37 min.',
       '20 min. per ep.', '2 hr. 15 min.', '26 min. per ep.',
       '28 min. per ep.', '21 min.', '46 min.', '15 min. per ep.',
       '2 hr. 5 min.', '29 min. per ep.', '33 min. per ep.',
       '1 hr. 24 min.', '48 min.', '1 hr. 19 min.', '25 min.',
       '1 hr. 30 min.', '1 hr. 1 min.', '4 min. per ep.',
       '7 min. per ep.', '1 hr. 46 min.', '18 min. per ep.',
       '1 hr. 41 min.', '11 min. per ep.', '13 min. per ep.', '55 min.',
       '1 hr. 21 min.', '1 hr. 2 min.', '37 min. per ep.',
       '42 min. per ep.', '37 min.', '19 min.', '1 hr. 

In [16]:
# Unknown to NaN in 'Duration' column
anime_df_cleaned['Duration'] = anime_df_cleaned['Duration'].replace('Unknown', pd.NA)

In [17]:
import pandas as pd
import re
import numpy as np

def convert_duration_to_seconds(duration):
    if pd.isna(duration):
        return np.nan

    hours_pattern = r"(\d+)\s*hr\.?"
    minutes_pattern = r"(\d+)\s*min\.?"
    seconds_pattern = r"(\d+)\s*sec\.?"
    
    hours = 0
    minutes = 0
    seconds = 0
    
    hours_match = re.search(hours_pattern, duration)
    minutes_match = re.search(minutes_pattern, duration)
    seconds_match = re.search(seconds_pattern, duration)

    if hours_match:
        hours = int(hours_match.group(1))
    if minutes_match:
        minutes = int(minutes_match.group(1))
    if seconds_match:
        seconds = int(seconds_match.group(1))
    
    total_seconds = hours * 3600 + minutes * 60 + seconds

    return total_seconds

In [18]:
# Convert 'Duration' to seconds
anime_df_cleaned['Duration'] = anime_df_cleaned['Duration'].apply(convert_duration_to_seconds)

print(anime_df_cleaned['Duration'].describe())

count    17007.000000
mean      1472.767096
std       1506.872993
min          3.000000
25%        300.000000
50%       1380.000000
75%       1620.000000
max      10020.000000
Name: Duration, dtype: float64


#### Loading data into DimAnime

In [19]:
anime_df_cleaned.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [20]:
# Insert data into stage.DimAnime after deleting existing data
cursor.execute("DELETE FROM stage.DimAnime;")
conn.commit()

for index, row in anime_df_cleaned.iterrows():
    cursor.execute("""
        INSERT INTO stage.DimAnime (
            MAL_ID, Name, English_name, Japanese_name, Type, Episodes, Aired, Premiered,
            Producers, Licensors, Studios, Source, Duration, Rating, Score, Ranked,
            Popularity, Members, Favorites, Watching, Completed, OnHold, Dropped, PlanToWatch
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        int(row['MAL_ID']), row['Name'], row['English name'], row['Japanese name'], row['Type'],
        int(row['Episodes']) if not pd.isna(row['Episodes']) else None,
        row['Aired'], row['Premiered'], row['Producers'], row['Licensors'],
        row['Studios'], row['Source'], int(row['Duration']) if not pd.isna(row['Duration']) else None,
        row['Rating'], float(row['Score']) if not pd.isna(row['Score']) else None,
        int(row['Ranked']) if not pd.isna(row['Ranked']) else None,
        int(row['Popularity']) if not pd.isna(row['Popularity']) else None,
        int(row['Members']) if not pd.isna(row['Members']) else None,
        int(row['Favorites']) if not pd.isna(row['Favorites']) else None,
        int(row['Watching']) if not pd.isna(row['Watching']) else None,
        int(row['Completed']) if not pd.isna(row['Completed']) else None,
        int(row['On-Hold']) if not pd.isna(row['On-Hold']) else None,
        int(row['Dropped']) if not pd.isna(row['Dropped']) else None,
        int(row['Plan to Watch']) if not pd.isna(row['Plan to Watch']) else None
    ))
conn.commit()


In [21]:
# Sanity check for DimAnime
cursor.execute("SELECT COUNT(*) FROM stage.DimAnime;")
rows_in_dimanime = cursor.fetchone()[0]
print(f"Sanity Check: Number of rows in DimAnime: {rows_in_dimanime}")
print("Rows in anime_df_cleaned:", len(anime_df_cleaned))
assert rows_in_dimanime == len(anime_df_cleaned)
print("Sanity check passed!")

Sanity Check: Number of rows in DimAnime: 17562
Rows in anime_df_cleaned: 17562
Sanity check passed!


#### Loading data into DimGenre and BridgeAnimeGenre

In [22]:
# Extract genres
genres_series = anime_df_cleaned['Genres'].dropna().str.split(',').explode().str.strip()
unique_genres = genres_series.unique()

# Insert into DimGenre after deleting existing data
cursor.execute("DELETE FROM stage.DimGenre;")
conn.commit()

genre_dict = {}
for genre in unique_genres:
    cursor.execute("INSERT INTO stage.DimGenre (genre_name) VALUES (?);", (genre,))
    genre_id = cursor.lastrowid
    genre_dict[genre] = genre_id
conn.commit()

# Create Bridge Table entries
bridge_genre_records = []
for index, row in anime_df_cleaned.iterrows():
    anime_id = int(row['MAL_ID'])
    genres = row['Genres']
    if pd.notnull(genres):
        genre_list = [g.strip() for g in genres.split(',')]
        for genre in genre_list:
            genre_id = genre_dict.get(genre)
            bridge_genre_records.append((anime_id, genre_id))

# Delete existing data in BridgeAnimeGenre
cursor.execute("DELETE FROM stage.BridgeAnimeGenre;")
conn.commit()

# Insert into BridgeAnimeGenre
insert_bridge_genre_query = """
INSERT INTO stage.BridgeAnimeGenre (anime_id, genre_id) VALUES (?, ?)
"""
cursor.executemany(insert_bridge_genre_query, bridge_genre_records)
conn.commit()


In [23]:
# Sanity check for BridgeAnimeGenre
cursor.execute("SELECT COUNT(*) FROM stage.BridgeAnimeGenre;")
rows_in_bridge_genre = cursor.fetchone()[0]
print(f"Sanity Check: Number of rows in BridgeAnimeGenre: {rows_in_bridge_genre}")
print("Rows in bridge_genre_records:", len(bridge_genre_records))
assert rows_in_bridge_genre == len(bridge_genre_records)
print("Sanity check passed!")

Sanity Check: Number of rows in BridgeAnimeGenre: 50198
Rows in bridge_genre_records: 50198
Sanity check passed!


#### Loading data into DimStudio and BridgeAnimeStudio

In [26]:
# Extract studios
studios_series = anime_df_cleaned['Studios'].dropna().str.split(',').explode().str.strip()
unique_studios = studios_series.unique()

# Insert into DimStudio after deleting existing data
cursor.execute("DELETE FROM stage.DimStudio;")
conn.commit()

studio_dict = {}
for studio in unique_studios:
    cursor.execute("INSERT INTO stage.DimStudio (studio_name) VALUES (?);", (studio,))
    studio_id = cursor.lastrowid
    studio_dict[studio] = studio_id
conn.commit()

# Create Bridge Table entries
bridge_studio_records = []
for index, row in anime_df_cleaned.iterrows():
    anime_id = int(row['MAL_ID'])
    studios = row['Studios']
    if pd.notnull(studios):
        studio_list = [s.strip() for s in studios.split(',')]
        for studio in studio_list:
            studio_id = studio_dict.get(studio)
            bridge_studio_records.append((anime_id, studio_id))

# Insert into BridgeAnimeStudio
cursor.execute("DELETE FROM stage.BridgeAnimeStudio;")
conn.commit()
insert_bridge_studio_query = """
INSERT INTO stage.BridgeAnimeStudio (anime_id, studio_id) VALUES (?, ?)
"""
cursor.executemany(insert_bridge_studio_query, bridge_studio_records)
conn.commit()


In [27]:

# Sanity check for BridgeAnimeStudio
cursor.execute("SELECT COUNT(*) FROM stage.BridgeAnimeStudio;")
rows_in_bridge_studio = cursor.fetchone()[0]
print(f"Sanity Check: Number of rows in BridgeAnimeStudio: {rows_in_bridge_studio}")
print("Rows in bridge_studio_records:", len(bridge_studio_records))
assert rows_in_bridge_studio == len(bridge_studio_records)
print("Sanity check passed!")

Sanity Check: Number of rows in BridgeAnimeStudio: 11295
Rows in bridge_studio_records: 11295
Sanity check passed!


### anime_with_synopsis.csv

#### Exploring anime_with_synopsis.csv

In [28]:
# anime_with_synopsis.csv
anime_with_synopsis_df = pd.read_csv('raw/anime_with_synopsis.csv')
print("Columns in anime_with_synopsis_df:")
print(anime_with_synopsis_df.columns)

Columns in anime_with_synopsis_df:
Index(['MAL_ID', 'Name', 'Score', 'Genres', 'sypnopsis'], dtype='object')


In [29]:
# Analyze the data
analysis_anime_with_synopsis_df = pd.DataFrame({
    'Data Type': anime_with_synopsis_df.dtypes,
    'Null': anime_with_synopsis_df.isnull().sum(),
    'NAN': anime_with_synopsis_df.isna().sum(),
    'Unknown': anime_with_synopsis_df.isin(['Unknown']).sum(),
    'Most Common Unique Values': [anime_with_synopsis_df[column].value_counts().index[:5].tolist() for column in anime_with_synopsis_df.columns],
})

analysis_anime_with_synopsis_df

,Data Type,Null,NAN,Unknown,Most Common Unique Values
MAL_ID,int64,0,0,0,"[48492, 1, 5, 6, 7]"
Name,object,0,0,0,[Maou Gakuin no Futekigousha: Shijou Saikyou n...
Score,object,0,0,5123,"[Unknown, 6.45, 6.31, 6.52, 6.48]"
Genres,object,0,0,63,"[Music, Comedy, Kids, Kids, Music, Dementia]"
sypnopsis,object,8,8,0,[No synopsis information has been added to thi...


#### Cleaning anime_with_synopsis.csv

In [30]:
# Handle Unknown values in Score and Genres
anime_with_synopsis_df.replace({'Genres': {'Unknown': np.nan}}, inplace=True)
anime_with_synopsis_df.replace({'Score': {'Unknown': np.nan}}, inplace=True)    

# Handle Numeric values
anime_with_synopsis_df['Score'] = pd.to_numeric(anime_with_synopsis_df['Score'], errors='coerce')

# Handle Synopsis column
anime_with_synopsis_df.rename(columns={'sypnopsis': 'synopsis'}, inplace=True)
placeholder_text = "No synopsis information has been added to this title."
anime_with_synopsis_df.replace({'synopsis': {placeholder_text: np.nan}}, inplace=True)

# Drop duplicates and NaN values in critical columns
anime_with_synopsis_df.drop_duplicates(inplace=True)
anime_with_synopsis_df.dropna(subset=['MAL_ID', 'Name'], inplace=True)


print(anime_with_synopsis_df.info())
anime_with_synopsis_df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16214 entries, 0 to 16213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MAL_ID    16214 non-null  int64  
 1   Name      16214 non-null  object 
 2   Score     11091 non-null  float64
 3   Genres    16151 non-null  object 
 4   synopsis  16206 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 633.5+ KB
None


MAL_ID         0
Name           0
Score       5123
Genres        63
synopsis       8
dtype: int64

### animelist.csv

#### Exploring animelist.csv

In [31]:
import pandas as pd

# animelist_df = pd.read_csv('raw/animelist.csv', low_memory=True)


In [32]:
# # Animelist_df
# analysis_animelist_df = pd.DataFrame({
#     'Data Type': animelist_df.dtypes,
#     'Null': animelist_df.isnull().sum(),
#     'NAN': animelist_df.isna().sum(),
#     'Unknown': animelist_df.isin(['Unknown']).sum(),
#     'Most Common Unique Values': [animelist_df[column].value_counts().index[:5].tolist() for column in animelist_df.columns],
# })

# analysis_animelist_df


In [33]:
# Total unique ratings
# animelist_df['rating'].nunique()

#### Reading, Cleaning and Loading unique user IDs into `DimUser` using chunking

Read `animelist.csv` in chunks to extract unique user IDs.



In [34]:
import pandas as pd

chunk_size = 100000 

unique_user_ids = set()
animelist_chunks = pd.read_csv('raw/animelist.csv', chunksize=chunk_size, usecols=['user_id'], low_memory=True)

for chunk in animelist_chunks:
    chunk = chunk.dropna(subset=['user_id'])
    unique_user_ids.update(chunk['user_id'].astype(int).unique())

# Prepare data for insertion into stage.DimUser
user_records = [(int(user_id),) for user_id in unique_user_ids]

# Insert into DimUser after deleting existing data
cursor.execute("DELETE FROM stage.DimUser;")
conn.commit()
cursor.executemany("INSERT INTO stage.DimUser (user_id) VALUES (?);", user_records)
conn.commit()


In [35]:

# Sanity check for DimUser
cursor.execute("SELECT COUNT(*) FROM stage.DimUser;")
rows_in_dimuser = cursor.fetchone()[0]
print(f"Sanity Check: Number of rows in DimUser: {rows_in_dimuser}")
print("Unique user IDs collected from animelist_df:", len(unique_user_ids))
assert rows_in_dimuser == len(unique_user_ids)
print("Sanity check passed!")


Sanity Check: Number of rows in DimUser: 325770
Unique user IDs collected from animelist_df: 325770
Sanity check passed!


#### Loading data into `FactUserAnimeInteractions` using chunking

Read `animelist.csv` in chunks, clean each chunk, and insert the data into the `FactUserAnimeInteractions` table.



In [37]:
import pandas as pd

chunk_size = 150000
total_rows_inserted = 0

animelist_chunks = pd.read_csv('raw/animelist.csv', chunksize=chunk_size, low_memory=True)

for chunk in animelist_chunks:
    
    chunk.drop_duplicates(inplace=True)
    
    # Drop rows with missing necessary values
    chunk.dropna(subset=['user_id', 'anime_id', 'watching_status'], inplace=True)
    
    # Convert numerical columns to ints
    chunk[['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes']] = chunk[['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes']].astype(int)
    
    # Prepare records and query for insertion
    fact_records = chunk[['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes']].values.tolist()
    
    insert_fact_query = """
    INSERT INTO stage.FactUserAnimeInteractions (
        user_id, anime_id, rating, watching_status, watched_episodes
    ) VALUES (%s, %s, %s, %s, %s)
    """
    
    try:
        cursor.executemany(insert_fact_query, fact_records)
        conn.commit()
        total_rows_inserted += len(fact_records)
    except mariadb.Error as e:
        print(f"Error inserting chunk: {e}")
        conn.rollback()
    
    print(f"Processed {total_rows_inserted} records.")

print(f"Total rows inserted: {total_rows_inserted}")


Processed 150000 records.
Processed 300000 records.
Processed 450000 records.
Processed 600000 records.
Processed 750000 records.
Processed 900000 records.
Processed 1050000 records.
Processed 1200000 records.
Processed 1350000 records.
Processed 1500000 records.
Processed 1650000 records.
Processed 1800000 records.
Processed 1950000 records.
Processed 2100000 records.
Processed 2250000 records.
Processed 2400000 records.
Processed 2550000 records.
Processed 2700000 records.
Processed 2850000 records.
Processed 3000000 records.
Processed 3150000 records.
Processed 3300000 records.
Processed 3450000 records.
Processed 3600000 records.
Processed 3750000 records.
Processed 3900000 records.
Processed 4050000 records.
Processed 4200000 records.
Processed 4350000 records.
Processed 4500000 records.
Processed 4650000 records.
Processed 4800000 records.
Processed 4950000 records.
Processed 5100000 records.
Processed 5250000 records.
Processed 5400000 records.
Processed 5550000 records.
Process

In [38]:

# Sanity check for FactUserAnimeInteractions
cursor.execute("SELECT COUNT(*) FROM stage.FactUserAnimeInteractions;")
rows_in_fact = cursor.fetchone()[0]
print(f"Sanity Check: Number of rows in FactUserAnimeInteractions: {rows_in_fact}")
print("Total rows inserted from animelist_df:", total_rows_inserted)
assert rows_in_fact == total_rows_inserted
print("Sanity check passed!")


Sanity Check: Number of rows in FactUserAnimeInteractions: 109224746
Total rows inserted from animelist_df: 109224746
Sanity check passed!


### rating_complete.csv

#### Exploring rating_complete.csv

In [39]:
# rating_complete.csv
rating_complete_df = pd.read_csv('raw/rating_complete.csv')
print("Columns in rating_complete_df:")
print(rating_complete_df.columns)


Columns in rating_complete_df:
Index(['user_id', 'anime_id', 'rating'], dtype='object')


In [40]:
# Analyze the data
analysis_rating_complete_df = pd.DataFrame({
    'Data Type': rating_complete_df.dtypes,
    'Null': rating_complete_df.isnull().sum(),
    'NAN': rating_complete_df.isna().sum(),
    'Unknown': rating_complete_df.isin(['Unknown']).sum(),
    'Most Common Unique Values': [rating_complete_df[column].value_counts().index[:5].tolist() for column in rating_complete_df.columns],
})

analysis_rating_complete_df

,Data Type,Null,NAN,Unknown,Most Common Unique Values
user_id,int64,0,0,0,"[189037, 162615, 68042, 283786, 259790]"
anime_id,int64,0,0,0,"[1535, 16498, 11757, 6547, 30276]"
rating,int64,0,0,0,"[8, 7, 9, 6, 10]"


In [41]:
# Ratings are between 1 and 10, ensure that
rating_complete_df = rating_complete_df[rating_complete_df['rating'].between(1, 10)]
rating_complete_df.drop_duplicates(inplace=True)

print(rating_complete_df.info())
rating_complete_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57633278 entries, 0 to 57633277
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 1.3 GB
None


user_id     0
anime_id    0
rating      0
dtype: int64

### watching_status.csv

#### Exploring watching_status.csv

In [42]:
watching_status_df = pd.read_csv('raw/watching_status.csv')
print("Columns in watching_status_df:")
print(watching_status_df.columns)

Columns in watching_status_df:
Index(['status', ' description'], dtype='object')


In [43]:
# Watching_status_df
analysis_watching_status_df = pd.DataFrame({
    'Data Type': watching_status_df.dtypes,
    'Null': watching_status_df.isnull().sum(),
    'NAN': watching_status_df.isna().sum(),
    'Unknown': watching_status_df.isin(['Unknown']).sum(),
    'Most Common Unique Values': [watching_status_df[column].value_counts().index[:5].tolist() for column in watching_status_df.columns],
})

analysis_watching_status_df

,Data Type,Null,NAN,Unknown,Most Common Unique Values
status,int64,0,0,0,"[1, 2, 3, 4, 6]"
description,object,0,0,0,"[Currently Watching, Completed, On Hold, Dropp..."


watching_status contains the information that relates status (int) to it's meaning description (string).

#### Clean watching_status.csv

There's not much to clean here.

In [44]:
# Drop duplicates
watching_status_df.drop_duplicates(inplace=True)


#### Loading data into DimWatchingStatus

In [45]:
status_records = watching_status_df.values.tolist()

insert_status_query = """
INSERT INTO stage.DimWatchingStatus (status, description) VALUES (%s, %s)
"""

cursor.executemany(insert_status_query, status_records)
conn.commit()


In [46]:

# Sanity check for DimWatchingStatus
cursor.execute("SELECT COUNT(*) FROM stage.DimWatchingStatus;")
rows_in_watchingstatus = cursor.fetchone()[0]
print(f"Sanity Check: Number of rows in DimWatchingStatus: {rows_in_watchingstatus}")
print("Rows in watching_status_df:", len(watching_status_df))
assert rows_in_watchingstatus == len(watching_status_df)
print("Sanity check passed!")


Sanity Check: Number of rows in DimWatchingStatus: 5
Rows in watching_status_df: 5
Sanity check passed!


# Hist Tables

## Create Hist tables with SCD Type 2 Fields

In [53]:
create_dimanime_hist = """
CREATE TABLE IF NOT EXISTS hist.DimAnime (
    anime_sk INT PRIMARY KEY AUTO_INCREMENT,
    MAL_ID INT,
    Name VARCHAR(255),
    English_name VARCHAR(255),
    Japanese_name VARCHAR(255),
    Type VARCHAR(50),
    Episodes INT,
    Aired VARCHAR(100),
    Premiered VARCHAR(50),
    Producers TEXT,
    Licensors TEXT,
    Studios TEXT,
    Source VARCHAR(50),
    Duration INT,
    Rating VARCHAR(50),
    Score FLOAT,
    Ranked INT,
    Popularity INT,
    Members INT,
    Favorites INT,
    Watching INT,
    Completed INT,
    OnHold INT,
    Dropped INT,
    PlanToWatch INT,
    effective_date DATE,
    expiry_date DATE,
    is_current BOOLEAN,
    UNIQUE (MAL_ID, effective_date)
);
"""
cursor.execute(create_dimanime_hist)
conn.commit()


In [54]:
cursor.execute("SHOW TABLES IN hist LIKE 'DimAnime';")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'hist.DimAnime' table created successfully.")

Sanity Check Passed: 'hist.DimAnime' table created successfully.


hist.DimAnime table has the fields needed for SCD Type 2: 
- `effective_date`
- `expiry_date`
- `is_current`


In [55]:
create_dimuser_hist = """
CREATE TABLE IF NOT EXISTS hist.DimUser (
    user_sk INT PRIMARY KEY AUTO_INCREMENT,
    user_id INT,
    effective_date DATE,
    expiry_date DATE,
    is_current BOOLEAN,
    UNIQUE (user_id, effective_date)
);
"""
cursor.execute(create_dimuser_hist)
conn.commit()


In [56]:
cursor.execute("SHOW TABLES IN hist LIKE 'DimUser';")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'hist.DimUser' table created successfully.")

Sanity Check Passed: 'hist.DimUser' table created successfully.


#### Create Fact Table

In [57]:
create_fact_hist = """
CREATE TABLE IF NOT EXISTS hist.FactUserAnimeInteractions (
    user_sk INT,
    anime_sk INT,
    rating INT,
    watching_status INT,
    watched_episodes INT,
    PRIMARY KEY (user_sk, anime_sk),
    FOREIGN KEY (user_sk) REFERENCES hist.DimUser(user_sk),
    FOREIGN KEY (anime_sk) REFERENCES hist.DimAnime(anime_sk)
);
"""
cursor.execute(create_fact_hist)
conn.commit()

In [58]:
cursor.execute("SHOW TABLES IN hist LIKE 'FactUserAnimeInteractions';")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'hist.FactUserAnimeInteractions' table created successfully.")

Sanity Check Passed: 'hist.FactUserAnimeInteractions' table created successfully.


With this we connect users to animes using foreign keys.

## Stored Procedures to Transfer Data from Stage to Hist (with handling for SCD Type 2)

##### hist.DimAnime

In [59]:
cursor.execute("DROP PROCEDURE IF EXISTS hist.SP_InsertUpdate_DimAnime;")
conn.commit()

sp_insertupdate_dimanime = """
CREATE PROCEDURE hist.SP_InsertUpdate_DimAnime()
BEGIN
    DECLARE done INT DEFAULT FALSE;
    DECLARE var_MAL_ID INT;
    DECLARE cur_stage CURSOR FOR SELECT MAL_ID FROM stage.DimAnime;
    DECLARE CONTINUE HANDLER FOR NOT FOUND SET done = TRUE;

    OPEN cur_stage;
    read_loop: LOOP
        FETCH cur_stage INTO var_MAL_ID;
        IF done THEN
            LEAVE read_loop;
        END IF;

        IF EXISTS (
            SELECT 1 FROM hist.DimAnime
            WHERE MAL_ID = var_MAL_ID AND is_current = TRUE
        ) THEN
            IF EXISTS (
                SELECT 1 FROM stage.DimAnime s
                JOIN hist.DimAnime h ON s.MAL_ID = h.MAL_ID
                WHERE s.MAL_ID = var_MAL_ID AND h.is_current = TRUE AND (
                    s.Name <> h.Name OR s.Type <> h.Type OR s.Score <> h.Score
                    -- Add other fields as needed
                )
            ) THEN
                -- Expire old record
                UPDATE hist.DimAnime
                SET expiry_date = CURDATE(), is_current = FALSE
                WHERE MAL_ID = var_MAL_ID AND is_current = TRUE;

                -- Add new record
                INSERT INTO hist.DimAnime (
                    MAL_ID, Name, English_name, Japanese_name, Type, Episodes, Aired, Premiered,
                    Producers, Licensors, Studios, Source, Duration, Rating, Score, Ranked,
                    Popularity, Members, Favorites, Watching, Completed, OnHold, Dropped, PlanToWatch,
                    effective_date, expiry_date, is_current
                )
                SELECT 
                    s.MAL_ID, s.Name, s.English_name, s.Japanese_name, s.Type, s.Episodes, s.Aired, s.Premiered,
                    s.Producers, s.Licensors, s.Studios, s.Source, s.Duration, s.Rating, s.Score, s.Ranked,
                    s.Popularity, s.Members, s.Favorites, s.Watching, s.Completed, s.OnHold, s.Dropped, s.PlanToWatch,
                    CURDATE(), NULL, TRUE
                FROM stage.DimAnime s
                WHERE s.MAL_ID = var_MAL_ID;
            END IF;
        ELSE
            -- Add new record
            INSERT INTO hist.DimAnime (
                MAL_ID, Name, English_name, Japanese_name, Type, Episodes, Aired, Premiered,
                Producers, Licensors, Studios, Source, Duration, Rating, Score, Ranked,
                Popularity, Members, Favorites, Watching, Completed, OnHold, Dropped, PlanToWatch,
                effective_date, expiry_date, is_current
            )
            SELECT 
                s.MAL_ID, s.Name, s.English_name, s.Japanese_name, s.Type, s.Episodes, s.Aired, s.Premiered,
                s.Producers, s.Licensors, s.Studios, s.Source, s.Duration, s.Rating, s.Score, s.Ranked,
                s.Popularity, s.Members, s.Favorites, s.Watching, s.Completed, s.OnHold, s.Dropped, s.PlanToWatch,
                CURDATE(), NULL, TRUE
            FROM stage.DimAnime s
            WHERE s.MAL_ID = var_MAL_ID;
        END IF;
    END LOOP;
    CLOSE cur_stage;
END;
"""

cursor.execute(sp_insertupdate_dimanime)
conn.commit()


In [60]:
cursor.execute("""
SELECT routine_name FROM information_schema.routines
WHERE routine_schema = 'hist' AND routine_name = 'SP_InsertUpdate_DimAnime';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: Stored Procedure 'SP_InsertUpdate_DimAnime' created successfully.")

Sanity Check Passed: Stored Procedure 'SP_InsertUpdate_DimAnime' created successfully.


##### hist.DimUser

In [61]:
cursor.execute("DROP PROCEDURE IF EXISTS hist.SP_InsertUpdate_DimUser;")
conn.commit()

sp_insertupdate_dimuser = """
CREATE PROCEDURE hist.SP_InsertUpdate_DimUser()
BEGIN
    INSERT INTO hist.DimUser (
        user_id, effective_date, expiry_date, is_current
    )
    SELECT 
        s.user_id, CURDATE(), NULL, TRUE
    FROM stage.DimUser s
    WHERE NOT EXISTS (
        SELECT 1 FROM hist.DimUser h
        WHERE s.user_id = h.user_id AND h.is_current = TRUE
    );
END;
"""

cursor.execute(sp_insertupdate_dimuser)
conn.commit()


In [62]:
cursor.execute("""
SELECT routine_name FROM information_schema.routines
WHERE routine_schema = 'hist' AND routine_name = 'SP_InsertUpdate_DimUser';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: Stored Procedure 'SP_InsertUpdate_DimUser' created successfully.")

Sanity Check Passed: Stored Procedure 'SP_InsertUpdate_DimUser' created successfully.


##### hist.FactUserAnimeInteractions

In [63]:
cursor.execute("DROP PROCEDURE IF EXISTS hist.SP_Insert_FactUserAnimeInteractions;")
conn.commit()

sp_insert_fact = """
CREATE PROCEDURE hist.SP_Insert_FactUserAnimeInteractions()
BEGIN
    INSERT INTO hist.FactUserAnimeInteractions (
        user_sk, anime_sk, rating, watching_status, watched_episodes
    )
    SELECT 
        du.user_sk,
        da.anime_sk,
        f.rating,
        f.watching_status,
        f.watched_episodes
    FROM stage.FactUserAnimeInteractions f
    JOIN hist.DimUser du ON f.user_id = du.user_id AND du.is_current = TRUE
    JOIN hist.DimAnime da ON f.anime_id = da.MAL_ID AND da.is_current = TRUE;
END
"""
cursor.execute(sp_insert_fact)
conn.commit()


In [64]:
cursor.execute("""
SELECT routine_name FROM information_schema.routines
WHERE routine_schema = 'hist' AND routine_name = 'SP_Insert_FactUserAnimeInteractions';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: Stored Procedure 'SP_Insert_FactUserAnimeInteractions' created successfully.")

Sanity Check Passed: Stored Procedure 'SP_Insert_FactUserAnimeInteractions' created successfully.


#### Execute SPs

In [65]:
cursor.execute("CALL hist.SP_InsertUpdate_DimUser();")
conn.commit()

cursor.execute("SELECT COUNT(*) FROM hist.DimUser WHERE is_current = TRUE;")
hist_dimuser_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM stage.DimUser;")
stage_dimuser_count = cursor.fetchone()[0]
print(f"Sanity Check: {hist_dimuser_count} records in hist.DimUser, {stage_dimuser_count} records in stage.DimUser.")
assert hist_dimuser_count == stage_dimuser_count
print("Sanity Check Passed: All users transferred to hist.DimUser.")

Sanity Check: 325770 records in hist.DimUser, 325770 records in stage.DimUser.
Sanity Check Passed: All users transferred to hist.DimUser.


In [66]:
cursor.execute("CALL hist.SP_InsertUpdate_DimAnime();")
conn.commit()

cursor.execute("SELECT COUNT(*) FROM hist.DimAnime WHERE is_current = TRUE;")
hist_dimanime_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM stage.DimAnime;")
stage_dimanime_count = cursor.fetchone()[0]
print(f"Sanity Check: {hist_dimanime_count} records in hist.DimAnime, {stage_dimanime_count} records in stage.DimAnime.")
assert hist_dimanime_count == stage_dimanime_count
print("Sanity Check Passed: All animes transferred to hist.DimAnime.")

Sanity Check: 17562 records in hist.DimAnime, 17562 records in stage.DimAnime.
Sanity Check Passed: All animes transferred to hist.DimAnime.


In [67]:
batch_size = 10000 
offset = 0

cursor.execute("SELECT COUNT(*) FROM stage.FactUserAnimeInteractions;")
total_rows_stage = cursor.fetchone()[0]

while offset < total_rows_stage:
    cursor.execute(f"""
        INSERT INTO hist.FactUserAnimeInteractions (user_sk, anime_sk, rating, watching_status, watched_episodes)
        SELECT du.user_sk, da.anime_sk, f.rating, f.watching_status, f.watched_episodes
        FROM stage.FactUserAnimeInteractions f
        JOIN hist.DimUser du ON f.user_id = du.user_id AND du.is_current = TRUE
        JOIN hist.DimAnime da ON f.anime_id = da.MAL_ID AND da.is_current = TRUE
        LIMIT {batch_size} OFFSET {offset};
    """)
    
    conn.commit()
    
    offset += batch_size
    print(f"Processed {min(offset, total_rows_stage)} / {total_rows_stage} records.")


In [ ]:

# Sanity check after batching
cursor.execute("SELECT COUNT(*) FROM hist.FactUserAnimeInteractions;")
hist_fact_count = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM stage.FactUserAnimeInteractions;")
stage_fact_count = cursor.fetchone()[0]

print(f"Sanity Check: {hist_fact_count} records in hist.FactUserAnimeInteractions, {stage_fact_count} records in stage.FactUserAnimeInteractions.")
assert hist_fact_count == stage_fact_count, "Mismatch between records in hist and stage."
print("Sanity Check Passed: All fact records transferred to hist.FactUserAnimeInteractions.")


## Triggers

#### Audit Table

In [ ]:
create_auditlog = """
CREATE TABLE IF NOT EXISTS hist.AuditLog (
    audit_id INT PRIMARY KEY AUTO_INCREMENT,
    table_name VARCHAR(100),
    operation VARCHAR(10),
    record_id INT,
    operation_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
cursor.execute(create_auditlog)
conn.commit()


In [ ]:
cursor.execute("SHOW TABLES IN hist LIKE 'AuditLog';")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'hist.AuditLog' table created successfully.")

#### After Insert Trigger

In [ ]:
cursor.execute("DROP TRIGGER IF EXISTS hist.DimAnime_AfterInsert;")
conn.commit()

trigger_after_insert = """
CREATE TRIGGER hist.DimAnime_AfterInsert
AFTER INSERT ON hist.DimAnime
FOR EACH ROW
BEGIN
    INSERT INTO hist.AuditLog (table_name, operation, record_id)
    VALUES ('DimAnime', 'INSERT', NEW.anime_sk);
END;
"""
cursor.execute(trigger_after_insert)
conn.commit()


In [ ]:
cursor.execute("""
SELECT TRIGGER_NAME FROM information_schema.triggers
WHERE TRIGGER_SCHEMA = 'hist' AND TRIGGER_NAME = 'DimAnime_AfterInsert';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'DimAnime_AfterInsert' trigger created successfully.")


#### After Update Trigger

In [ ]:
cursor.execute("DROP TRIGGER IF EXISTS hist.DimAnime_AfterUpdate;")
conn.commit()

trigger_after_update = """
CREATE TRIGGER hist.DimAnime_AfterUpdate
AFTER UPDATE ON hist.DimAnime
FOR EACH ROW
BEGIN
    INSERT INTO hist.AuditLog (table_name, operation, record_id)
    VALUES ('DimAnime', 'UPDATE', NEW.anime_sk);
END;
"""
cursor.execute(trigger_after_update)
conn.commit()


In [ ]:
cursor.execute("""
SELECT TRIGGER_NAME FROM information_schema.triggers
WHERE TRIGGER_SCHEMA = 'hist' AND TRIGGER_NAME = 'DimAnime_AfterUpdate';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'DimAnime_AfterUpdate' trigger created successfully.")

#### After Delete Trigger

In [ ]:
cursor.execute("DROP TRIGGER IF EXISTS hist.DimAnime_AfterDelete;")
conn.commit()

trigger_after_delete = """
CREATE TRIGGER hist.DimAnime_AfterDelete
AFTER DELETE ON hist.DimAnime
FOR EACH ROW
BEGIN
    INSERT INTO hist.AuditLog (table_name, operation, record_id)
    VALUES ('DimAnime', 'DELETE', OLD.anime_sk);
END;
"""
cursor.execute(trigger_after_delete)
conn.commit()


In [ ]:
cursor.execute("""
SELECT TRIGGER_NAME FROM information_schema.triggers
WHERE TRIGGER_SCHEMA = 'hist' AND TRIGGER_NAME = 'DimAnime_AfterDelete';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'DimAnime_AfterDelete' trigger created successfully.")

## Views

### Filter Views

#### FV1 - High-Scoring Anime

In [ ]:
cursor.execute("DROP VIEW IF EXISTS hist.View_HighScoreAnimes;")
conn.commit()

view_high_score_animes = """
CREATE VIEW hist.View_HighScoreAnimes AS
SELECT * FROM hist.DimAnime
WHERE Score >= 9.0 AND is_current = TRUE;
"""
cursor.execute(view_high_score_animes)
conn.commit()

In [ ]:
cursor.execute("""
SELECT TABLE_NAME FROM information_schema.VIEWS
WHERE TABLE_SCHEMA = 'hist' AND TABLE_NAME = 'View_HighScoreAnimes';
""")
result = cursor.fetchone()
assert result is not None, "Sanity Check Failed: 'View_HighScoreAnimes' view was not created."
print("Sanity Check Passed: 'View_HighScoreAnimes' view created successfully.")

In [ ]:
# Check if the view returns data
cursor.execute("SELECT COUNT(*) FROM hist.View_HighScoreAnimes;")
count = cursor.fetchone()[0]
assert count >= 0, "Sanity Check Failed: 'View_HighScoreAnimes' query failed."
print(f"'View_HighScoreAnimes' contains {count} records.")


#### FV2 - Recent Anime

In [ ]:
cursor.execute("DROP VIEW IF EXISTS hist.View_RecentAnimes;")
conn.commit()

view_recent_animes = """
CREATE VIEW hist.View_RecentAnimes AS
SELECT * FROM hist.DimAnime
WHERE effective_date >= DATE_SUB(CURDATE(), INTERVAL 1 YEAR) AND is_current = TRUE;
"""
cursor.execute(view_recent_animes)
conn.commit()

#### FV3 - Anime by Studio

In [ ]:
cursor.execute("DROP VIEW IF EXISTS hist.View_AnimesByStudio;")
conn.commit()

view_animes_by_studio = """
CREATE VIEW hist.View_AnimesByStudio AS
SELECT a.*
FROM hist.DimAnime a
JOIN stage.BridgeAnimeStudio bas ON a.MAL_ID = bas.anime_id
JOIN stage.DimStudio s ON bas.studio_id = s.studio_id
WHERE s.studio_name = 'Studio Ghibli' AND a.is_current = TRUE;
"""
cursor.execute(view_animes_by_studio)
conn.commit()

### Aggregate Views

#### AV1 - Avg Anime Score by Genre

In [ ]:
cursor.execute("DROP VIEW IF EXISTS hist.View_AverageScorePerGenre;")
conn.commit()

view_avg_score_genre = """
CREATE VIEW hist.View_AverageScorePerGenre AS
SELECT g.genre_name, AVG(a.Score) AS average_score
FROM hist.DimGenre g
JOIN stage.BridgeAnimeGenre bag ON g.genre_id = bag.genre_id
JOIN hist.DimAnime a ON bag.anime_id = a.MAL_ID
WHERE a.is_current = TRUE
GROUP BY g.genre_name;
"""
cursor.execute(view_avg_score_genre)
conn.commit()


In [ ]:
cursor.execute("""
SELECT TABLE_NAME FROM information_schema.VIEWS
WHERE TABLE_SCHEMA = 'hist' AND TABLE_NAME = 'View_AverageScorePerGenre';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'View_AverageScorePerGenre' view created successfully.")

#### AV2 - Total people in each watching status

In [ ]:
cursor.execute("DROP VIEW IF EXISTS hist.View_TotalMembersByStatus;")
conn.commit()

view_total_members_status = """
CREATE VIEW hist.View_TotalMembersByStatus AS
SELECT w.description, SUM(a.Members) AS total_members
FROM stage.DimWatchingStatus w
JOIN hist.FactUserAnimeInteractions f ON w.status = f.watching_status
JOIN hist.DimAnime a ON f.anime_sk = a.anime_sk
WHERE a.is_current = TRUE
GROUP BY w.description;
"""
cursor.execute(view_total_members_status)
conn.commit()


##### AV3 - Top Studios by Anime Count

In [ ]:
cursor.execute("DROP VIEW IF EXISTS hist.View_TopStudiosByAnimeCount;")
conn.commit()

view_top_studios = """
CREATE VIEW hist.View_TopStudiosByAnimeCount AS
SELECT s.studio_name, COUNT(bas.anime_id) AS anime_count
FROM stage.DimStudio s
JOIN stage.BridgeAnimeStudio bas ON s.studio_id = bas.studio_id
GROUP BY s.studio_name
ORDER BY anime_count DESC
LIMIT 5;
"""
cursor.execute(view_top_studios)
conn.commit()


## Stored Procedures for Tasks

#### SP 1 - Refresh Genre Statistics

In [ ]:
create_genrestats = """
CREATE TABLE IF NOT EXISTS hist.GenreStats (
    genre_name VARCHAR(100),
    anime_count INT,
    average_score FLOAT,
    PRIMARY KEY (genre_name)
);
"""
cursor.execute(create_genrestats)
conn.commit()

In [ ]:
cursor.execute("SHOW TABLES IN hist LIKE 'GenreStats';")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: 'hist.GenreStats' table created successfully.")

In [ ]:
cursor.execute("DROP PROCEDURE IF EXISTS hist.SP_RefreshGenreStats;")
conn.commit()

sp_refresh_genre_stats = """
CREATE PROCEDURE hist.SP_RefreshGenreStats()
BEGIN
    DELETE FROM hist.GenreStats;
    
    INSERT INTO hist.GenreStats (genre_name, anime_count, average_score)
    SELECT g.genre_name, COUNT(a.anime_sk), AVG(a.Score)
    FROM hist.DimGenre g
    JOIN stage.BridgeAnimeGenre bag ON g.genre_id = bag.genre_id
    JOIN hist.DimAnime a ON bag.anime_id = a.MAL_ID
    WHERE a.is_current = TRUE
    GROUP BY g.genre_name;
END;
"""
cursor.execute(sp_refresh_genre_stats)
conn.commit()

In [ ]:
cursor.execute("""
SELECT ROUTINE_NAME FROM information_schema.ROUTINES
WHERE ROUTINE_SCHEMA = 'hist' AND ROUTINE_NAME = 'SP_RefreshGenreStats';
""")
result = cursor.fetchone()
assert result is not None
print("Sanity Check Passed: Stored Procedure 'SP_RefreshGenreStats' created successfully.")

#### SP 2 - Refresh Anime Popularity Rank

In [ ]:
cursor.execute("DROP PROCEDURE IF EXISTS hist.SP_UpdatePopularityRankings;")
conn.commit()

sp_update_popularity = """
DELIMITER $$

CREATE PROCEDURE hist.SP_UpdatePopularityRankings()
BEGIN
    SET @rank = 0;
    
    UPDATE hist.DimAnime a
    JOIN (
        SELECT anime_sk, (@rank := @rank + 1) AS popularity_rank
        FROM hist.DimAnime
        WHERE is_current = TRUE
        ORDER BY Members DESC
    ) ranks ON a.anime_sk = ranks.anime_sk
    SET a.Popularity = ranks.popularity_rank;
END $$

DELIMITER ;
"""
cursor.execute(sp_update_popularity)
conn.commit()


#### SP 3 - Archive for Expired Records

In [ ]:
# Create hist.Archived_DimAnime table
create_archived_dimanime = """
CREATE TABLE IF NOT EXISTS hist.Archived_DimAnime LIKE hist.DimAnime;
"""
cursor.execute(create_archived_dimanime)
conn.commit()

In [ ]:
cursor.execute("DROP PROCEDURE IF EXISTS hist.SP_ArchiveExpiredDimensions;")
conn.commit()

sp_archive_expired = """
DELIMITER $$

CREATE PROCEDURE hist.SP_ArchiveExpiredDimensions()
BEGIN
    INSERT INTO hist.Archived_DimAnime
    SELECT * FROM hist.DimAnime
    WHERE is_current = FALSE AND expiry_date < DATE_SUB(CURDATE(), INTERVAL 1 YEAR);
    
    DELETE FROM hist.DimAnime
    WHERE is_current = FALSE AND expiry_date < DATE_SUB(CURDATE(), INTERVAL 1 YEAR);
END $$

DELIMITER ;
"""
cursor.execute(sp_archive_expired)
conn.commit()

# End

In [ ]:
cursor.close()
conn.close()